# AutoML

##  Load Azure subscription details

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

subscription_id = config['AZURE']['SUBSCRIPTION_ID']
resource_group = config['AZURE']['RESOURCE_GROUP']
region = 'westeurope'

## Connect to the workspace

In [ ]:
name = 'AMLSLearnworkspace'

In [ ]:
from azureml.core import Workspace, Experiment, Run

ws = Workspace.get(name=name,
                   subscription_id=subscription_id,
                   resource_group=resource_group)

print('Done')

In [ ]:
# Load the diabetes dataset, a well-known built-in small dataset that comes with scikit-learn.
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

# More info on the dataset: https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset
X, y = load_diabetes(return_X_y = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print('Done')

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Create an experiment

In [ ]:
from azureml.core.experiment import Experiment

experiment = Experiment(workspace = ws, name = "automl-experiment")

print('Done')

## Define the machine learning objective and constraints 

In [ ]:
# from azureml.core.compute import ComputeTarget

# compute_target = ComputeTarget.list(ws)[0]
# compute_target

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
import logging

automl_config = AutoMLConfig(task = 'regression',
                  iteration_timeout_minutes = 10,
                  iterations = 10,
                  primary_metric = 'spearman_correlation',
                  n_cross_validations = 5,
                  debug_log = 'automl.log',
                  verbosity = logging.INFO,
                  #compute_target = compute_target,
                  X = X_train, 
                  y = y_train)

print('Done')

## Start the AutoML process

In [ ]:
local_run = experiment.submit(automl_config, show_output = True)

## Retrieve the best model

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

In [ ]:
best_run, fitted_model = local_run.get_output()

## Test the best model

In [ ]:
y_pred_train = fitted_model.predict(X_train)
y_residual_train = y_train - y_pred_train
y_pred_test = fitted_model.predict(X_test)
y_residual_test = y_test - y_pred_test

In [ ]:
y_residual_test